# Analytic Question 1
What is the distribution of democracy across the world, and how does it vary between different aspects of democracy and over time?


## Data preparation

In [ ]:
# Load packages
import altair as alt
import pandas as pd
import numpy as np
from vega_datasets import data

# Load data 
geo_data = pd.read_csv("../../../data/processed/geo_data.csv")

# Load Country ID for mapping 
country_ids = pd.read_csv('https://raw.githubusercontent.com/kemiolamudzengi/dsci-320-datasets/main/country-ids-and-continents.csv')
country_ids = country_ids.rename(columns={"Country": "country",
                                          "Continent": "continent"})

# Changing country names to match the names in geo_data (missing 16 countries)
country_ids['country'] = country_ids['country'].replace({
    'Democratic Republic of Congo': 'Democratic Republic of the Congo',
    'United States': 'United States of America',
    "Cote d'Ivoire": "Ivory Coast",
    'Turkey': 'Türkiye',
    "Korea, Democratic People's Republic of": 'North Korea',
    'Myanmar': 'Burma/Myanmar',
    'Congo': 'Republic of the Congo',
    'Macedonia, the former Yugoslav Republic of': 'North Macedonia',
    'Gambia': 'The Gambia',
    'Taiwan, Province of China':'Taiwan',
    'Swaziland': 'Eswatini'
})

relevant_country_names = country_ids["country"]

# Merge Country IDs to geo_data, creating view I datasat, df1. 
df1 = geo_data.query(  
    'country in @relevant_country_names'
    #  only countries that have IDs
).merge(  # Add the ID column by joining with country_ids
    country_ids,  # Automatically joins on 'country' column
)

df1.sample(10)

In [ ]:
# List of countries in geo_data, that doesn't have an equivalent in the ID mapping data (and therefore not included in the df1 dataframe).
missing_countries = [
    "Bahrain",
    "Barbados",
    "Cape Verde",
    "Comoros",
    "Hong Kong",
    "Kosovo",
    "Maldives",
    "Malta",
    "Mauritius",
    "Palestine/Gaza",
    "Palestine/West Bank",
    "Sao Tome and Principe",
    "Seychelles",
    "Singapore",
    "Somaliland",
    "Zanzibar"
]
print(len(missing_countries))


## Creating global variables

In [ ]:
# Democracy indices
indices = [
    "mean_democracy",
    "electoral_democracy",
    "liberal_democracy",
    "participatory_democracy",
    "deliberative_democracy",
    "egalitarian_democracy"
]

# Pretty labels for dropdown
index_labels = {
    "mean_democracy": "Overall Democracy (mean)",
    "electoral_democracy": "Electoral Democracy",
    "liberal_democracy": "Liberal Democracy",
    "participatory_democracy": "Participatory Democracy",
    "deliberative_democracy": "Deliberative Democracy",
    "egalitarian_democracy": "Egalitarian Democracy"
}

# Creating dropdown menu of democracy index
dropdown = alt.binding_select(
    options=list(index_labels.values()),
    name="Select index: "
)

index_param = alt.param(
    name="selected_label",
    bind=dropdown,
    value=list(index_labels.values())[0]
)

# Creating click-selection of countries
country_select = alt.selection_point(
    fields=['country'],
    toggle=True, 
    empty='none')

# Creating color scheme 
color_scale = alt.Scale(scheme="blues", domain=[0,1])

# Customizing non-blue colors
nonblue_colors = [
    "#1b9e77",  
    "#d95f02",  
    "#7570b3",  
    "#e7298a",  
    "#66a61e",
    "#e6ab02"  
]


## Visualization 1: Choropleth map of Democracy in 2024

In [ ]:
alt.data_transformers.enable("vegafusion")

# Load world country boundaries from built-in TopoJSON file
world_map = alt.topo_feature(data.world_110m.url, 'countries')

# Create the choropleth map
democracy_map = alt.Chart(world_map).mark_geoshape()

# Join the data + encode
democracy_map = democracy_map.transform_lookup(
    # Join map data with democracy data using country IDs
    lookup='id',                    
    from_=alt.LookupData(df1[df1['year'] == 2024], 'ID', ['country', 'continent'] + indices) # Filtering data to the year 2024. 
).transform_calculate(
    selected_score=(
        "selected_label === 'Overall Democracy (mean)' ? datum.mean_democracy : "
        "selected_label === 'Electoral Democracy' ? datum.electoral_democracy : "
        "selected_label === 'Liberal Democracy' ? datum.liberal_democracy : "
        "selected_label === 'Participatory Democracy' ? datum.participatory_democracy : "
        "selected_label === 'Deliberative Democracy' ? datum.deliberative_democracy : "
        "datum.egalitarian_democracy"
    )
).encode(
        tooltip=[
            alt.Tooltip('country:N', title='Country'),
            alt.Tooltip('continent:N', title='Continent'),
            alt.Tooltip('selected_score:Q', title='Score', format='.3f')],
        color=alt.Color(
            'selected_score:Q',
            title='Democracy Score',
            scale=color_scale,
            legend=alt.Legend(
                orient='right',
                direction='vertical',
                titleFontWeight='bold',
                gradientLength=350,   
                gradientThickness=20, 
                padding=10)),
        stroke=alt.condition(
            country_select,
            alt.value('black'),
            alt.value(None))   
).project(
    'equalEarth'
).properties(
    width=900,                      
    height=400,
    title={
        "text": "Global Democracy Levels",
        "subtitle": "Graph is based on Feb 2024 data and shows the spatial distribution of the selected democracy index."
    }
).add_params(
    index_param,
    country_select
)


democracy_map

## Visualization 2: Bar-graph of regional means

In [ ]:
bar_chart = (
    alt.Chart(df1[df1['year'] == 2024]) # Filtering data to the year 2024. Matching the data in the map. 
    .mark_bar(color ='darkgrey')
    .encode(
        x=alt.X('continent:N', title=None , sort='-y'),
        y=alt.Y('mean(selected_score):Q', title='Mean Democracy Score'),
        tooltip=[
            alt.Tooltip('continent:N', title='Continent'),
            alt.Tooltip('mean(selected_score):Q',
                        title='Mean Score',
                        format=".3f")
        ]
    )
    .transform_calculate(
        selected_score=(
            "selected_label === 'Overall Democracy (mean)' ? datum.mean_democracy : "
            "selected_label === 'Electoral Democracy' ? datum.electoral_democracy : "
            "selected_label === 'Liberal Democracy' ? datum.liberal_democracy : "
            "selected_label === 'Participatory Democracy' ? datum.participatory_democracy : "
            "selected_label === 'Deliberative Democracy' ? datum.deliberative_democracy : "
            "datum.egalitarian_democracy"
        )
    )
    .properties(
        width=150,
        height=200,
        title={
            "text": "Average Democracy Score by Continent",
            "subtitle": "Graph is based on Feb 2024 data."
    }
    )
    .add_params(index_param)
)


## Visualization 3: Line Chart 

In [ ]:
alt.data_transformers.enable("vegafusion")

line_base = (
    alt.Chart(df1)    
    .transform_calculate(
        selected_score=(
            "selected_label === 'Overall Democracy (mean)' ? datum.mean_democracy : "
            "selected_label === 'Electoral Democracy' ? datum.electoral_democracy : "
            "selected_label === 'Liberal Democracy' ? datum.liberal_democracy : "
            "selected_label === 'Participatory Democracy' ? datum.participatory_democracy : "
            "selected_label === 'Deliberative Democracy' ? datum.deliberative_democracy : "
            "datum.egalitarian_democracy"))
    .mark_line(point=False)
    .encode(
        x=alt.X(
            'year:O',
            title="Year",
            axis=alt.Axis(
                values=[year for year in range(1900, 2025) if year % 5 == 0], # Customizing the x-axis to show every fifth year. 
                grid=True,
                gridColor='lightgray',
                gridWidth=1,
                labelAngle=0,
                labelExpr="datum.value % 5 === 0 ? datum.value : ''")),
        y=alt.Y(
            'selected_score:Q',
            title="Democracy Score",
            scale=alt.Scale(domain=(0, 1)),                     # Fixing the y-axis to span from 0 to 1. 
            axis=alt.Axis(
                values=[i/5 for i in range(0, 11)],
                grid=True,
                gridColor='lightgray',
                gridWidth=1)),
        color=alt.Color(
            'country:N', 
            title="Country", 
            scale=alt.Scale(range=nonblue_colors), 
            legend=alt.Legend(
                symbolType="square",
                symbolSize=200)),
        detail='country:N',
        tooltip=[
            alt.Tooltip('country:N', title = 'Country'), 
            alt.Tooltip('continent:N', title='Continent'),
            alt.Tooltip('year:O', title ='Year'),
            alt.Tooltip('selected_score:Q', title='Score', format= '.3f')])
    .transform_filter(country_select)
    .properties(
        width=650,
        height=200,
        title={'text': "Democracy Trends for Selected Countrie(s)",
               'subtitle': "Maximum data coverage ranges from 1900 to 2024"})
    .add_params(index_param, country_select)
)

select_layer = line_base.mark_line(strokeWidth=10, opacity=0).add_params(country_select)

line_chart= select_layer+line_base


# View I

In [ ]:
# Enable large dataset support
alt.data_transformers.enable("vegafusion")

# Defining spacers for use in the dashboard.
spacer = alt.Chart(pd.DataFrame({'x':[0]})).mark_text(text='').properties(height=10)
vspacer = alt.Chart(pd.DataFrame({'x':[0]})).mark_text(text='').properties(height=30)


# Create dashboard 
dashboard = alt.vconcat(
    democracy_map,
    spacer,
    bar_chart |vspacer| line_chart
).properties(
    title={
        "text": "Democracy Worldwide",
        "subtitle": "Spatial Distribution, Regional Averages, and Temporal Dynamics of Selected Democracy Index.",
        "anchor": "middle",   
        "font": "Helvetica",
        "fontSize": 38,
        "fontWeight": "bold",
        "subtitleFont": "Helvetica",
        "subtitleFontSize": 16,
        "subtitleColor": "gray",
        "offset": 20          
    },
    padding={"right": 100, "left": 20, "top": 20, "bottom": 20}
).configure_legend(
    labelFont='Helvetica',
    titleFont='Helvetica',
    orient='right',
    titleFontSize=12,
    labelFontSize=12
).configure_title(
    font='Helvetica',
    fontSize=16,
    subtitleFontSize= 14,
    anchor='start',     # 'start' = left, 'middle' = center, 'end' = right
    color='black'
).configure_axis(
    labelFont='Helvetica',
    titleFont='Helvetica'
).configure_view(
    stroke=None
)



dashboard